In [43]:
import pandas as pd
import numpy as np
from pyspark.sql import functions as f
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

StatementMeta(spark001, 2, 44, Finished, Available)

In [26]:
df = spark.read.parquet('abfss://files@datalakesu4wxcm.dfs.core.windows.net/synapse/workspaces/synapsesu4wxcm/warehouse/Stock_price/')

StatementMeta(spark001, 2, 27, Finished, Available)

In [27]:
NVDA_DF = df.filter(f.col('symbol')=='NVDA')
pdf = NVDA_DF.toPandas().sort_values(by='date',ascending=False)

StatementMeta(spark001, 2, 28, Finished, Available)

In [38]:
close_prices = pdf['close'].values
close_prices = close_prices.reshape(-1, 1)

StatementMeta(spark001, 2, 39, Finished, Available)

In [39]:
# Feature scaling
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices)

# Create the training data set
training_data_len = int(np.ceil(len(scaled_data) * 0.8))
train_data = scaled_data[0:int(training_data_len), :]

# Create the training data sequences
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the training data for LSTM model
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

# Create the testing data set
test_data = scaled_data[training_data_len - 60:, :]

x_test = []
y_test = close_prices[training_data_len:, :]

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)


StatementMeta(spark001, 2, 40, Finished, Available)

39/39 [==============================] - 2s 17ms/step


In [55]:
predictions

StatementMeta(spark001, 2, 56, Finished, Available)

array([[11.4691305],
       [11.705694 ],
       [11.821334 ],
       ...,
       [31.985983 ],
       [31.251762 ],
       [29.601564 ]], dtype=float32)

In [46]:
pdf['close'][:training_data_len]

StatementMeta(spark001, 2, 47, Finished, Available)

5947    135.580002
5948    130.979996
5949    131.880005
5950    129.610001
5951    125.199997
           ...    
1064     14.230000
1065     14.320000
1066     14.530000
1067     14.510000
1068     15.350000
Name: close, Length: 4958, dtype: float32

In [50]:
valid

StatementMeta(spark001, 2, 51, Finished, Available)

1069                                                       15.26
1070                                                        15.2
1071                                                       15.55
1072                                                       15.12
1073                                                       14.52
                                     ...                        
6193                                                   29.190001
6194                                                   27.440001
6195                                                        25.0
6196                                                        23.5
Predictions    [[11.4691305], [11.705694], [11.821334], [11.9...
Name: close, Length: 1240, dtype: object

In [52]:
print(type(pdf['close'][:training_data_len]))

StatementMeta(spark001, 2, 53, Finished, Available)

<class 'pandas.core.series.Series'>


In [54]:
# Plot the data
train= pdf['close'][:training_data_len]
valid= pdf['close'][training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price USD')
plt.plot(train)
plt.plot(valid[['close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()


StatementMeta(spark001, 2, 55, Finished, Available)

KeyError: "['close'] not in index"

In [ ]:
# Predict future prices for the next 30 days
last_60_days = scaled_data[-60:]
future_predictions = []

for _ in range(30):
    X_test = np.reshape(last_60_days, (1, last_60_days.shape[0], 1))
    pred_price = model.predict(X_test)
    future_predictions.append(pred_price[0, 0])
    last_60_days = np.append(last_60_days, pred_price)[1:]

future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Display the future predictions
print(future_predictions)